#### Import Lib


In [20]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#### Đọc dữ liệu


In [21]:
df_real = pd.read_csv('../Data/input_data.csv')
df = df_real.copy()

In [22]:
# Hàm loại bỏ giá trị ngoại lai
def remove_outliers(df, columns=None):
    """
    Loại bỏ giá trị ngoại lai khỏi các cột được chỉ định trong DataFrame.
    - df: DataFrame đầu vào
    - columns: Danh sách các cột cần kiểm tra (nếu None, sẽ kiểm tra tất cả các cột số)
    """
    if columns is None:
        columns = df.select_dtypes(include=['float', 'int']).columns  # Chỉ lấy các cột số

    for col in columns:
        # Tính IQR (Interquartile Range)
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        # Xác định phạm vi hợp lệ
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Loại bỏ ngoại lai
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

# Áp dụng hàm
df = remove_outliers(df)

In [23]:
df.columns

Index(['Brand', 'Processor_Name', 'Processor_Brand', 'RAM', 'RAM_TYPE',
       'Display_type', 'GPU', 'GPU_Brand', 'SSD', 'OS', 'VRAM', 'Price_VND'],
      dtype='object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3034 entries, 0 to 3915
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Brand            3034 non-null   object
 1   Processor_Name   3034 non-null   object
 2   Processor_Brand  3034 non-null   object
 3   RAM              3034 non-null   int64 
 4   RAM_TYPE         3034 non-null   object
 5   Display_type     3034 non-null   object
 6   GPU              3034 non-null   object
 7   GPU_Brand        3034 non-null   object
 8   SSD              3034 non-null   int64 
 9   OS               3034 non-null   object
 10  VRAM             3034 non-null   int64 
 11  Price_VND        3034 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 308.1+ KB


#### Train - Test Split


In [25]:
# Giả sử dữ liệu của bạn có tên là df
X = df.drop('Price_VND', axis=1)  # Các biến độc lập
y = df['Price_VND']  # Biến mục tiêu

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Xác định biến số và biến phân loại
# Xác định các biến số và biến phân loại
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

#### Tạo Pipeline tiền xử lý


In [27]:
# Tạo pipeline cho tiền xử lý
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Kết hợp các bước tiền xử lý vào ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

#### Xây dựng Pipeline cho mô hình hồi quy


In [28]:
# Danh sách các mô hình hồi quy
models = {
    'XGBRegressor': XGBRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'KNeighborsRegressor': KNeighborsRegressor()
}

# Huấn luyện và đánh giá mô hình
results = {}

for model_name, model in models.items():
    # Tạo pipeline cho mô hình
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    
    # Huấn luyện mô hình
    pipeline.fit(X_train, y_train)
    
    # Dự đoán trên tập train và tập test
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # # Đánh giá mô hình trên tập train
    # train_mae = mean_absolute_error(y_train, y_train_pred)
    # train_mse = mean_squared_error(y_train, y_train_pred)
    # train_r2 = r2_score(y_train, y_train_pred)
    
    # Đánh giá mô hình trên tập test
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # # Lưu kết quả đánh giá
    # results[model_name] = {
    #     'Train MAE': train_mae, 'Test MAE': test_mae,
    #     'Train MSE': train_mse, 'Test MSE': test_mse,
    #     'Train R2': train_r2,   'Test R2': test_r2
    # }
    results[model_name] = {
        'MAE': test_mae,
        'MSE': test_mse,
        'R2': test_r2
    }

# Hiển thị kết quả
results_df = pd.DataFrame(results).T
results_df


,MAE,MSE,R2
XGBRegressor,2.207033e+06,9.270887e+12,0.770631
DecisionTreeRegressor,2.716211e+06,1.538571e+13,0.619346
RandomForestRegressor,2.365790e+06,1.087625e+13,0.730913
LinearRegression,2.317085e+06,9.790896e+12,0.757766
Ridge,2.308289e+06,9.788094e+12,0.757835
KNeighborsRegressor,2.676205e+06,1.397831e+13,0.654166


In [29]:
# # Danh sách các mô hình hồi quy
# models = {
#     'XGBRegressor': XGBRegressor(),
#     'DecisionTreeRegressor': DecisionTreeRegressor(),
#     'RandomForestRegressor': RandomForestRegressor(),
#     'LinearRegression': LinearRegression(),
#     'Ridge': Ridge(),
#     'KNeighborsRegressor': KNeighborsRegressor()
# }

# # Huấn luyện và đánh giá mô hình
# results = {}

# for model_name, model in models.items():
#     # Tạo pipeline cho mô hình
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                ('regressor', model)])
    
#     # Huấn luyện mô hình
#     pipeline.fit(X_train, y_train)
    
#     # Dự đoán trên tập kiểm tra
#     y_pred = pipeline.predict(X_test)
    
#     # Đánh giá mô hình
#     mae = mean_absolute_error(y_test, y_pred)
#     mse = mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)
    
#     # Lưu kết quả đánh giá
#     results[model_name] = {'MAE': mae, 'MSE': mse, 'R2': r2}

# # Hiển thị kết quả
# results_df = pd.DataFrame(results).T
# results_df


Kết quả:

- Mô hình tốt nhất: XGBRegressor có R² cao nhất (81.8%) và MAE thấp, làm cho nó trở thành mô hình tốt nhất trong bộ so sánh này.
- Mô hình kém nhất: KNeighborsRegressor có R² thấp nhất (78.4%) và MSE cao nhất, cho thấy nó không phù hợp với dữ liệu này so với các mô hình khác.

Hướng phát triển:

- Tối ưu tham số cho mô hình XGBRegressor


#### Tối ưu tham số cho mô hình XGBRegressor


In [30]:
# Danh sách một số param cần lưu ý
xgb_params = {
    'n_estimators',        # Số lượng cây quyết định
    'learning_rate',       # Tốc độ học (eta)
    'max_depth',             # Độ sâu tối đa của cây
    'min_child_weight',      # Trọng số tối thiểu của mỗi nút lá
    'subsample',           # Tỷ lệ mẫu huấn luyện mỗi cây
    'colsample_bytree',    # Tỷ lệ đặc trưng được chọn để huấn luyện mỗi cây
    'gamma',                 # Mức độ phạt để ngừng tạo nhánh
    'scale_pos_weight',      # Tỷ lệ giữa các lớp trong bài toán mất cân bằng
    'objective',  # Mục tiêu hồi quy
    'eval_metric',      # Chỉ số đánh giá
    'tree_method',      # Phương pháp xây dựng cây
    'lambda',              # Regularization L2 (Ridge)
    'alpha'               # Regularization L1 (Lasso)
}


In [31]:
# Xây dựng Pipeline riêng cho mô hình XGBRegressor
XGB_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor())  # Mô hình XGBRegressor
    ]
)

# Huấn luyện lại mô hình với tham số mặc định
XGB_pipeline.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình với tham số mặc định
y_pred = XGB_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# In ra các kết quả đánh giá và tham số mặc định
print("Kết quả đánh giá với tham số mặc định:")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")
# XGB_pipeline.named_steps['regressor'].get_params()

# Tạo DataFrame từ y_test và y_pred
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})


# In ra các tham số có trong danh sách xgb_params
print("\nCác tham số của XGBRegressor trong danh sách cần lưu ý:")
xgb_model_params = XGB_pipeline.named_steps['regressor'].get_params()

for param in xgb_params:
    if param in xgb_model_params:
        print(f"{param}: {xgb_model_params[param]}")


Kết quả đánh giá với tham số mặc định:
MAE: 2207033.0366556835
MSE: 9270887279339.035
R²: 0.7706311941146851

Các tham số của XGBRegressor trong danh sách cần lưu ý:
min_child_weight: None
n_estimators: None
tree_method: None
objective: reg:squarederror
colsample_bytree: None
max_depth: None
gamma: None
learning_rate: None
eval_metric: None
scale_pos_weight: None
subsample: None


In [32]:
# Hàm dự đoán giá bán laptop
def predict_laptop_price(brand, processor_name, processor_brand, ram, ram_type, display_type, gpu, gpu_brand, ssd, os, vram):
    # Tạo DataFrame cho dữ liệu nhập
    input_data = pd.DataFrame({
        'Brand': [brand],
        'Processor_Name': [processor_name],
        'Processor_Brand': [processor_brand],
        'RAM': [ram],
        'RAM_TYPE': [ram_type],
        'Display_type': [display_type],
        'GPU': [gpu],
        'GPU_Brand': [gpu_brand],
        'SSD': [ssd],
        'OS': [os],
        'VRAM': [vram]
    })
    
    # Dự đoán giá bán
    predicted_price = XGB_pipeline.predict(input_data)
    return predicted_price[0]

# Ví dụ sử dụng hàm
predicted_price = predict_laptop_price(
    brand='Lenovo', processor_name='AMD Octa-Core Ryzen 7', processor_brand='AMD', ram=16, ram_type='DDR5', 
    display_type='LCD', gpu='Geforce RTX 4060', gpu_brand='NVIDIA', ssd=512, os='Windows 11', vram=8
)

print(f"Giá bán dự đoán của laptop là: {predicted_price} VND")

Giá bán dự đoán của laptop là: 21275050.0 VND


In [33]:
df[df['Processor_Name'].str.contains('Ryzen 7')]['Processor_Name'].unique()


array(['AMD Octa-Core Ryzen 7', 'AMD Octa-Core Ryzen 7 Processor',
       'AMD Quad-Core Ryzen 7', 'AMD Octa-Core Ryzen 7 Pro'], dtype=object)

In [34]:
results_df

,Actual,Predicted
3699,24897000,21028382.0
2903,10267500,10410747.0
3105,24120000,18003924.0
490,11610000,13445709.0
1079,17697000,17854176.0
...,...,...
3442,11817000,13825256.0
3520,24297000,18837386.0
3176,13052100,17303804.0
799,12075000,14382863.0


In [35]:
# Cross-validation bằng Stratified K-Fold với 5 folds
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
mae_scores = []
mse_scores = []
r2_scores = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    XGB_pipeline.fit(X_train, y_train)
    y_pred = XGB_pipeline.predict(X_test)
    mae_scores.append(mean_absolute_error(y_test, y_pred))
    mse_scores.append(mean_squared_error(y_test, y_pred))
    r2_scores.append(r2_score(y_test, y_pred))

print("MAE của các fold:", mae_scores)
print("MAE trung bình:", np.mean(mae_scores))
print("MSE của các fold:", mse_scores)
print("MSE trung bình:", np.mean(mse_scores))
print("R2 của các fold:", r2_scores)
print("R2 trung bình:", np.mean(r2_scores))

c:\Users\ME\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MAE của các fold: [np.float64(2305017.1256177924), np.float64(2282111.1116144974), np.float64(2238267.601317957), np.float64(2215548.644151565), np.float64(2342674.5969471945)]
MAE trung bình: 2276723.815929801
MSE của các fold: [np.float64(10115230802821.014), np.float64(9116828838102.896), np.float64(9448447219801.31), np.float64(9815679278304.12), np.float64(11350938850093.977)]
MSE trung bình: 9969424997824.662
R2 của các fold: [0.7405433058738708, 0.7714547514915466, 0.7585083246231079, 0.7438485622406006, 0.7135012149810791]
R2 trung bình: 0.745571231842041


In [36]:
# Cross-validation bằng Stratified K-Fold với 10 folds
num_folds = 10
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
mae_scores = []
mse_scores = []
r2_scores = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    XGB_pipeline.fit(X_train, y_train)
    y_pred = XGB_pipeline.predict(X_test)
    mae_scores.append(mean_absolute_error(y_test, y_pred))
    mse_scores.append(mean_squared_error(y_test, y_pred))
    r2_scores.append(r2_score(y_test, y_pred))

print("MAE của các fold:", mae_scores)
print("MAE trung bình:", np.mean(mae_scores))
print("MSE của các fold:", mse_scores)
print("MSE trung bình:", np.mean(mse_scores))
print("R2 của các fold:", r2_scores)
print("R2 trung bình:", np.mean(r2_scores))

c:\Users\ME\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


MAE của các fold: [np.float64(2502645.208881579), np.float64(2371425.193256579), np.float64(2133481.4021381577), np.float64(2266159.5822368423), np.float64(2238507.8836633665), np.float64(2277445.021452145), np.float64(2131609.405940594), np.float64(2250998.004950495), np.float64(2279397.0495049506), np.float64(2154286.3044554456)]
MAE trung bình: 2260595.5056480155
MSE của các fold: [np.float64(11154400185230.9), np.float64(10306521056305.877), np.float64(8806007882122.416), np.float64(10514067531022.9), np.float64(9977808730193.814), np.float64(8907102108153.684), np.float64(7644140773766.689), np.float64(9136482190695.346), np.float64(10539579643800.152), np.float64(9737775679978.158)]
MSE trung bình: 9672388578126.994
R2 của các fold: [0.7410118579864502, 0.7549107074737549, 0.7663607597351074, 0.7053282260894775, 0.7383351922035217, 0.7429229617118835, 0.7958922386169434, 0.7747670412063599, 0.7427266836166382, 0.7631112337112427]
R2 trung bình: 0.7525366902351379
